# Assignment 5

## 1.复习课上内容， 阅读相应论文。

## 2. 回答以下理论题目

### 2. 1.  What is autoencoder?

对原数据进行加工，从中提取出主要特征，是一种无监督学习过程

### 2. 2. What are the differences between greedy search and beam search?

贪心搜索是在句子的每一个词都是相应的最大概率
集束搜索返回top（n）个结果，在下一个单词的选取的时候，根据第一个单词的条件概率选取top（n）个，逐步选取下一个单词

### 2. 3. What is the intuition of attention mechanism?

注意力机制为了解决句子中前面的重要单词可能出现信息损失的情况，编码时给出一个隐状态，解码时根据前一个隐状态和编码时对应的隐状态给出结果

### 2. 4. What is the disadvantage of word embeding introduced in previous lectures ?

只能给出分类或固定长度的结果，seq2seq可以给出序列结果，也就是变长结果

### 2. 5. Briefly describe what is self-attention and what is multi-head attention?

self-attention计算每个单词和其他单词之间的联系，得到一个注意力分数；multi-head-attention是由多个self-attention组合而成的，结果可能和self-attention不一样。

## 3. 中英文自动翻译模型的构建（使用encoder-decoder模型）

![](https://media.geeksforgeeks.org/wp-content/uploads/seq2seq.png)

### 3.1 [中英文翻译数据集下载](http://www.manythings.org/anki/)
找到Chinese (Mandarin) - English cmn-eng.zip (22075条中英文翻译)

### 3.2  数据处理：encoder的输入，decoder的输入与输出
1，句子转换为one-hot编码     
2，LSTM需要的三维输入[n_samples, timestamp, one-hot feature]

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

### 3.3 encoder-decoder模型的搭建
1，模型训练    
2，模型推理     
3，模型预测，展示结果   

In [ ]:
#encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

# decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)


encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)


    target_seq = np.zeros((1, 1, num_decoder_tokens))

    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

loss: 0.1474 - accuracy: 0.9713 - val_loss: 2.3929 - val_accuracy: 0.6940

In [ ]:
for seq_index in range(1, 100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

'''
-
Input sentence: Hi.
Decoded sentence: 你好。

-
Input sentence: Run.
Decoded sentence: 你用跑的。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I try.
Decoded sentence: 让我来。

-
Input sentence: I won!
Decoded sentence: 我赢。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 乾杯!

-
Input sentence: Got it?
Decoded sentence: 你懂了吗？

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 跳进来。

-
Input sentence: I quit.
Decoded sentence: 我退出。

-
Input sentence: I'm OK.
Decoded sentence: 我沒事。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: Really?
Decoded sentence: 你确定？

-
Input sentence: Try it.
Decoded sentence: 试试吧。

-
Input sentence: We try.
Decoded sentence: 我们来试试。

-
Input sentence: Why me?
Decoded sentence: 为什么是我？

-
Input sentence: Ask Tom.
Decoded sentence: 去问汤姆。

-
Input sentence: Awesome!
Decoded sentence: 好棒！

-
Input sentence: Be calm.
Decoded sentence: 冷静点。

-
Input sentence: Be fair.
Decoded sentence: 公平点。

-
Input sentence: Be kind.
Decoded sentence: 友善点。

-
Input sentence: Be nice.
Decoded sentence: 友善点。

-
Input sentence: Be nice.
Decoded sentence: 友善点。

-
Input sentence: Call me.
Decoded sentence: 联系我。

-
Input sentence: Call us.
Decoded sentence: 联系我们。

-
Input sentence: Come in.
Decoded sentence: 进来。

-
Input sentence: Get Tom.
Decoded sentence: 找到汤姆。

-
Input sentence: Get out!
Decoded sentence: 滾！

-
Input sentence: Get out!
Decoded sentence: 滾！

-
Input sentence: Go away!
Decoded sentence: 走開！

-
Input sentence: Go away!
Decoded sentence: 走開！

-
Input sentence: Go away.
Decoded sentence: 走開！

-
Input sentence: Go home.
Decoded sentence: 回家吧。

-
Input sentence: Goodbye!
Decoded sentence: 再见！

-
Input sentence: Goodbye!
Decoded sentence: 再见！

-
Input sentence: Hang on!
Decoded sentence: 坚持。

-
Input sentence: Hang on!
Decoded sentence: 坚持。

-
Input sentence: Hang on.
Decoded sentence: 坚持。

-
Input sentence: He came.
Decoded sentence: 他来了。

-
Input sentence: He runs.
Decoded sentence: 他跑。

-
Input sentence: Help me.
Decoded sentence: 帮我一下。

-
Input sentence: Help us.
Decoded sentence: 帮帮我们吧！

-
Input sentence: Hold on.
Decoded sentence: 坚持。

-
Input sentence: Hug Tom.
Decoded sentence: 坚持。

-
Input sentence: Hug Tom.
Decoded sentence: 坚持。

-
Input sentence: I agree.
Decoded sentence: 我同意。

-
Input sentence: I'm ill.
Decoded sentence: 我生病了。

-
Input sentence: I'm old.
Decoded sentence: 我老了。

-
Input sentence: I'm wet.
Decoded sentence: 我个子高興。

-
Input sentence: It's OK.
Decoded sentence: 没关系。

-
Input sentence: It's me.
Decoded sentence: 是我。

-
Input sentence: Join us.
Decoded sentence: 来加入我们吧。

-
Input sentence: Keep it.
Decoded sentence: 留着吧。

-
Input sentence: Kiss me.
Decoded sentence: 我走吧。

-
Input sentence: Perfect!
Decoded sentence: 完美！

-
Input sentence: See you.
Decoded sentence: 再见！

-
Input sentence: Shut up!
Decoded sentence: 友善点。

-
Input sentence: Skip it.
Decoded sentence: 不管它。

-
Input sentence: Take it.
Decoded sentence: 拿走吧。

-
Input sentence: Wake up!
Decoded sentence: 醒醒！

-
Input sentence: Wash up.
Decoded sentence: 去清洗一下。

-
Input sentence: We know.
Decoded sentence: 我们知道。

-
Input sentence: Welcome.
Decoded sentence: 欢迎。

-
Input sentence: Who won?
Decoded sentence: 谁赢了？

-
Input sentence: Why not?
Decoded sentence: 为什么不？

-
Input sentence: You run.
Decoded sentence: 你跑。

-
Input sentence: Back off.
Decoded sentence: 往后退点。

-
Input sentence: Be still.
Decoded sentence: 静静的，别动。

-
Input sentence: Beats me.
Decoded sentence: 我一无所知。

-
Input sentence: Cuff him.
Decoded sentence: 把他铐上。

-
Input sentence: Drive on.
Decoded sentence: 往前开。

-
Input sentence: Get away!
Decoded sentence: 走開！

-
Input sentence: Get away!
Decoded sentence: 走開！

-
Input sentence: Get down!
Decoded sentence: 們出去！

-
Input sentence: Get lost!
Decoded sentence: 滾！

-
Input sentence: Get real.
Decoded sentence: 醒醒吧。

-
Input sentence: Good job!
Decoded sentence: 做得好！

-
Input sentence: Good job!
Decoded sentence: 做得好！

-
Input sentence: Grab Tom.
Decoded sentence: 抓住汤姆。

-
Input sentence: Grab him.
Decoded sentence: 抓住他。

-
Input sentence: Have fun.
Decoded sentence: 玩得開心。

-
Input sentence: He tries.
Decoded sentence: 他来试试。

-
Input sentence: How cute!
Decoded sentence: 多可爱啊！

-
Input sentence: Humor me.
Decoded sentence: 你就随了我的意吧。

-
Input sentence: Hurry up.
Decoded sentence: 快点！

-
Input sentence: Hurry up.
Decoded sentence: 快点！

-
Input sentence: Hurry up.
Decoded sentence: 快点！

-
Input sentence: I forgot.
Decoded sentence: 我忘了。

-
Input sentence: I resign.
Decoded sentence: 我放弃。

-
Input sentence: I'll pay.
Decoded sentence: 我來付錢。

-
Input sentence: I'm busy.
Decoded sentence: 我很忙。

-
Input sentence: I'm cold.
Decoded sentence: 我老了。

-
Input sentence: I'm fine.
Decoded sentence: 我很好。

-
Input sentence: I'm full.
Decoded sentence: 我吃飽了。
'''

![](https://stickershop.line-scdn.net/stickershop/v1/product/3624648/LINEStorePC/main.png;compress=true)